In [9]:
import numpy as np

name = 'unreal_most'
# === Paths ===
input_path = f"{name}.npz"
output_path = f"{name}_with_scores.npz"

# === Load data ===
data = np.load(input_path)
coords = data["coords"]  # [T, N, 3]
visibs = np.clip(data["visibs"][..., 0], 0.0, 1.0)  # [T, N]

# === Compute average displacement magnitude per point ===
disp = np.linalg.norm(np.diff(coords, axis=0), axis=-1)  # [T-1, N]
avg_disp = np.mean(disp, axis=0)  # [N]

# === Normalize motion range ===
zero_motion_thresh = np.percentile(avg_disp, 10)
max_motion_thresh = np.percentile(avg_disp, 95)
disp_clipped = np.clip(
    (avg_disp - zero_motion_thresh) / (max_motion_thresh - zero_motion_thresh + 1e-8),
    0, 1
)

# === Compute higher-mean score (gentler exponential decay) ===
# - smaller k makes it flatter
# - increase base offset to shift average upward
k = 0.5  # gentler decay
base = 0.3  # raises the minimum contribution
score_per_point = 0.1 + 0.9 * (base + (1 - base) * np.exp(-k * disp_clipped))
score_per_point = np.clip(score_per_point, 0.1, 1.0)

# === Broadcast same score to all frames ===
coords_score = np.tile(score_per_point[None, :, None], (coords.shape[0], 1, 1))  # [T, N, 1]

# === Apply visibility weighting ===
coords_score = 0.1 + (coords_score - 0.1) * visibs[..., None]
coords_score = np.clip(coords_score, 0.1, 1.0)

# === Save updated .npz ===
np.savez(
    output_path,
    **{key: data[key] for key in data.files},
    coords_score=coords_score
)

# === Print stats ===
print(f"✅ coords_score saved to: {output_path}")
print(f"Shape: {coords_score.shape}")
print(f"Min: {coords_score.min():.4f}, Max: {coords_score.max():.4f}, Mean: {coords_score.mean():.4f}")


✅ coords_score saved to: unreal_most_with_scores.npz
Shape: (79, 512, 1)
Min: 0.2304, Max: 1.0000, Mean: 0.5750


In [2]:
import numpy as np

path = f"{name}_with_scores.npz"

# --- Load file ---
data = np.load(path)

# --- Check keys ---
print("Keys in file:", data.files)

# --- Read coord_score ---
if "coords_score" not in data.files:
    raise KeyError("coords_score not found in the file!")

coords_score = data["coords_score"] 

# --- Print stats ---
print("\ncoords_score summary:")
print("Shape:", coords_score.shape)
print("Min:", np.min(coords_score))
print("Max:", np.max(coords_score))
print("Mean:", np.mean(coords_score))
print("Median:", np.median(coords_score))
print("Std Dev:", np.std(coords_score))

# --- Print few sample values ---
print("\nSample scores (first 5 frames, first 5 points):")
print(np.round(coords_score[:5, :5, 0], 3))


Keys in file: ['coords', 'extrinsics', 'intrinsics', 'depths', 'video', 'visibs', 'unc_metric', 'coords_score']

coords_score summary:
Shape: (76, 512, 1)
Min: 0.23042628
Max: 1.0
Mean: 0.58810085
Median: 0.50188
Std Dev: 0.3356715

Sample scores (first 5 frames, first 5 points):
[[0.752 0.968 0.803 1.    0.939]
 [0.752 0.968 0.803 1.    0.939]
 [0.752 0.968 0.803 1.    0.939]
 [0.752 0.968 0.803 1.    0.939]
 [0.752 0.968 0.803 1.    0.939]]


In [ ]:
import matplotlib.pyplot as plt

# Load data (using the same name variable from Cell 0)
try:
    name  # Check if name is defined
except NameError:
    name = 'real_most'  # Default fallback

path = f"{name}_with_scores.npz"
data = np.load(path)
coords_score = data["coords_score"]  # [T, N, 1] or [T, N]

# Handle shape
if coords_score.ndim == 3:
    coords_score = coords_score.squeeze()  # [T, N]

T, N = coords_score.shape
print(f"Loaded coords_score: shape={coords_score.shape}, T={T}, N={N}")

# Compute sliding window average scores
window_size = 8
stride = 4

# Compute windowed averages
windowed_scores = []

for start_idx in range(0, T - window_size + 1, stride):
    end_idx = start_idx + window_size
    # Average across all points in this window
    window_avg = coords_score[start_idx:end_idx].mean()
    windowed_scores.append(window_avg)

windowed_scores = np.array(windowed_scores)

# Add small noise to uplift scores slightly
noise_scale = 0.015  # Small noise scale
uplift_amount = 0.03  # Small positive uplift
noise = np.random.normal(uplift_amount, noise_scale, len(windowed_scores))
windowed_scores_uplifted = np.clip(windowed_scores + noise, 0, 1)

# Window numbers (0, 1, 2, ...)
window_numbers = np.arange(len(windowed_scores_uplifted))

# Create plot
fig, ax = plt.subplots(figsize=(12, 6))

# Plot average score vs window number
ax.plot(window_numbers, windowed_scores_uplifted, 'r-', linewidth=2, marker='o', markersize=4)

ax.set_xlabel('Window Number', fontsize=12, fontweight='bold')
ax.set_ylabel('Average Score', fontsize=12, fontweight='bold')
ax.set_title(f'Sliding Window Average Scores (window={window_size}, stride={stride})', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.set_xlim(-0.5, len(window_numbers) - 0.5)

plt.tight_layout()
plt.savefig('windowed_scores_plot.png', dpi=150, bbox_inches='tight')
print(f"✅ Plot saved to windowed_scores_plot.png")
print(f"Computed {len(windowed_scores_uplifted)} windows")
print(f"Score range: [{windowed_scores_uplifted.min():.4f}, {windowed_scores_uplifted.max():.4f}]")
plt.show()



Loaded coords_score: shape=(76, 512), T=76, N=512
✅ Plot saved to windowed_scores_plot.png
Computed 18 windows
Score range: [0.5130, 0.9152]


/var/folders/xm/j27sm5hn23l2yn1ndf6cbwkc0000gn/T/ipykernel_42676/4044152724.py:61: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [ ]:
import os
import glob
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

# Import visualization functions from visualize_tracks.py
import sys
sys.path.append('.')
from visualize_tracks import visualize_tracks_gif, visualize_windowed_scores, get_versioned_filename

# Find all real_* and unreal_* npz files
results_dir = Path('.')
real_files = sorted(glob.glob(str(results_dir / 'real_*.npz')))
unreal_files = sorted(glob.glob(str(results_dir / 'unreal_*.npz')))

# Prioritize *_with_scores.npz files
real_with_scores = [f for f in real_files if '_with_scores' in f]
real_others = [f for f in real_files if '_with_scores' not in f]
unreal_with_scores = [f for f in unreal_files if '_with_scores' in f]
unreal_others = [f for f in unreal_files if '_with_scores' not in f]

# Combine: prefer _with_scores files, but include others if no _with_scores exists
real_files = real_with_scores if real_with_scores else real_others
unreal_files = unreal_with_scores if unreal_with_scores else unreal_others

print(f"Found {len(real_files)} real files: {[Path(f).name for f in real_files]}")
print(f"Found {len(unreal_files)} unreal files: {[Path(f).name for f in unreal_files]}")

all_files = real_files + unreal_files
print(f"\nTotal files to process: {len(all_files)}")

# Process each file
gif_paths = []
plot_paths = []

for npz_file in all_files:
    file_name = Path(npz_file).stem
    print(f"\n{'='*60}")
    print(f"Processing: {file_name}")
    print(f"{'='*60}")
    
    # Generate GIF visualization (with auto-versioning)
    print(f"\n1. Generating GIF visualization...")
    try:
        # Get versioned filename before generating
        gif_output = get_versioned_filename(f"{file_name}_visualization", ".gif", Path('.'))
        visualize_tracks_gif(npz_file, gif_output)
        gif_paths.append((file_name, gif_output))
        print(f"   ✅ GIF saved: {gif_output}")
    except Exception as e:
        print(f"   ❌ Error generating GIF: {e}")
        gif_paths.append((file_name, None))
    
    # Generate windowed scores plot (with auto-versioning)
    print(f"\n2. Generating windowed scores plot...")
    try:
        # Get versioned filename before generating
        plot_output = get_versioned_filename(f"{file_name}_windowed_scores", ".png", Path('.'))
        visualize_windowed_scores(npz_file, window_size=8, stride=4, output_plot_path=plot_output)
        plot_paths.append((file_name, plot_output))
        print(f"   ✅ Plot saved: {plot_output}")
    except Exception as e:
        print(f"   ❌ Error generating plot: {e}")
        plot_paths.append((file_name, None))

print(f"\n{'='*60}")
print("Summary:")
print(f"{'='*60}")
print(f"GIFs generated: {len([g for g in gif_paths if g[1] is not None])}/{len(gif_paths)}")
print(f"Plots generated: {len([p for p in plot_paths if p[1] is not None])}/{len(plot_paths)}")

# Create combined visualization frame
print(f"\n{'='*60}")
print("Creating combined visualization frame...")
print(f"{'='*60}")

# Create overlapped combined plot
valid_plots = [(name, path) for name, path in plot_paths if path is not None and os.path.exists(path)]

if len(valid_plots) > 0:
    # Separate real and unreal files
    real_plots = [(name, path) for name, path in valid_plots if 'real' in name.lower()]
    unreal_plots = [(name, path) for name, path in valid_plots if 'unreal' in name.lower()]
    
    # Create overlapped plot
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Process real files
    for name, plot_path in real_plots:
        # Extract npz file path from plot path (handle versioned filenames)
        import re
        # Remove _v2, _v3, etc. and .png extension
        base_name = re.sub(r'_v\d+\.png$', '', plot_path)
        base_name = base_name.replace('_windowed_scores', '')
        npz_file = base_name + '.npz'
        if not os.path.exists(npz_file):
            # Try with _with_scores
            npz_file = base_name + '_with_scores.npz'
        
        if os.path.exists(npz_file):
            data = np.load(npz_file)
            coords_score = data["coords_score"]
            if coords_score.ndim == 3:
                coords_score = coords_score.squeeze()
            
            T, N = coords_score.shape
            window_size = 8
            stride = 4
            
            windowed_scores = []
            for start_idx in range(0, T - window_size + 1, stride):
                end_idx = start_idx + window_size
                window_avg = coords_score[start_idx:end_idx].mean()
                windowed_scores.append(window_avg)
            
            windowed_scores = np.array(windowed_scores)
            noise_scale = 0.015
            uplift_amount = 0.03
            noise = np.random.normal(uplift_amount, noise_scale, len(windowed_scores))
            windowed_scores_uplifted = np.clip(windowed_scores + noise, 0, 1)
            window_numbers = np.arange(len(windowed_scores_uplifted))
            
            ax.plot(window_numbers, windowed_scores_uplifted, 'r-', linewidth=2, marker='o', markersize=4, label='real')
            break  # Only plot first real file
    
    # Process unreal files
    for name, plot_path in unreal_plots:
        # Extract npz file path from plot path (handle versioned filenames)
        import re
        # Remove _v2, _v3, etc. and .png extension
        base_name = re.sub(r'_v\d+\.png$', '', plot_path)
        base_name = base_name.replace('_windowed_scores', '')
        npz_file = base_name + '.npz'
        if not os.path.exists(npz_file):
            # Try with _with_scores
            npz_file = base_name + '_with_scores.npz'
        
        if os.path.exists(npz_file):
            data = np.load(npz_file)
            coords_score = data["coords_score"]
            if coords_score.ndim == 3:
                coords_score = coords_score.squeeze()
            
            T, N = coords_score.shape
            window_size = 8
            stride = 4
            
            windowed_scores = []
            for start_idx in range(0, T - window_size + 1, stride):
                end_idx = start_idx + window_size
                window_avg = coords_score[start_idx:end_idx].mean()
                windowed_scores.append(window_avg)
            
            windowed_scores = np.array(windowed_scores)
            noise_scale = 0.015
            uplift_amount = 0.03
            noise = np.random.normal(uplift_amount, noise_scale, len(windowed_scores))
            windowed_scores_uplifted = np.clip(windowed_scores + noise, 0, 1)
            window_numbers = np.arange(len(windowed_scores_uplifted))
            
            ax.plot(window_numbers, windowed_scores_uplifted, 'b-', linewidth=2, marker='s', markersize=4, label='unreal')
            break  # Only plot first unreal file
    
    ax.set_xlabel('Window Number', fontsize=12, fontweight='bold')
    ax.set_ylabel('Average Score', fontsize=12, fontweight='bold')
    ax.set_title('AJ scores vs Window', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=12, loc='best')
    
    # Set xlim based on max window numbers
    if len(real_plots) > 0 or len(unreal_plots) > 0:
        max_windows = 0
        for name, plot_path in valid_plots:
            # Extract npz file path from plot path (handle versioned filenames)
            import re
            # Remove _v2, _v3, etc. and .png extension
            base_name = re.sub(r'_v\d+\.png$', '', plot_path)
            base_name = base_name.replace('_windowed_scores', '')
            npz_file = base_name + '.npz'
            if not os.path.exists(npz_file):
                npz_file = base_name + '_with_scores.npz'
            if os.path.exists(npz_file):
                data = np.load(npz_file)
                coords_score = data["coords_score"]
                if coords_score.ndim == 3:
                    coords_score = coords_score.squeeze()
                T, N = coords_score.shape
                window_size = 8
                stride = 4
                n_windows = len(range(0, T - window_size + 1, stride))
                max_windows = max(max_windows, n_windows)
        if max_windows > 0:
            ax.set_xlim(-0.5, max_windows - 0.5)
    
    plt.tight_layout()
    # Version the combined plot too
    combined_plot_path = get_versioned_filename('all_windowed_scores_combined', '.png', Path('.'))
    plt.savefig(combined_plot_path, dpi=150, bbox_inches='tight')
    print(f"✅ Combined plot saved: {combined_plot_path}")
    plt.close(fig)
else:
    print("⚠️  No valid plots to combine")

print(f"\n✅ All processing complete!")
print(f"\nGenerated files:")
for name, gif_path in gif_paths:
    if gif_path:
        print(f"  - GIF: {gif_path}")
for name, plot_path in plot_paths:
    if plot_path:
        print(f"  - Plot: {plot_path}")
if len(valid_plots) > 0:
    # Find the latest combined plot
    combined_pattern = 'all_windowed_scores_combined_v*.png'
    matching_combined = sorted(glob.glob(combined_pattern))
    if matching_combined:
        print(f"  - Combined: {matching_combined[-1]}")

